# Рекомендация тарифов

После предобработки в прошлом проекте на основе данных с тарифами "Смарт" и "Ультра" необходимо построить модель для задачи классификации тарифа. Основная метрика - *accuracy*.

## Изучение данных

In [10]:
# загружаем необходимые библиотеки
import pandas as pd 
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split # разделение выборки
from sklearn.metrics import mean_squared_error # mse
from sklearn.dummy import DummyClassifier #в 5 бонусный пункт

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
display(df)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


**Вывод**

Получили данные с информацией поведения пользователей за 1 месяц.
- сalls — количество звонков
- minutes — суммарная длительность звонков в минутах
- messages — количество sms-сообщений
- mb_used — израсходованный интернет-трафик в Мб
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0)

## Деление данных на выборки

In [4]:
# создадим два датафрейма с признаками и ответами
features = df.loc[:,'calls':'mb_used']
target = df['is_ultra']
display(features.head())
display(target.head())

,calls,minutes,messages,mb_used
0,40.0,311.90,83.0,19915.42
1,85.0,516.75,56.0,22696.96
2,77.0,467.66,86.0,21060.45
3,106.0,745.53,81.0,8437.39
4,66.0,418.74,1.0,14502.75


0    0
1    0
2    0
3    1
4    0
Name: is_ultra, dtype: int64

In [5]:
# разобьем данные на обучающую, валидационную и тестовую выборки, применив метод train_ttest_split дважды
df_train_test, df_valid = train_test_split(df, test_size=0.2, train_size=0.8, random_state=12345) 
df_train, df_test = train_test_split(df_train_test, test_size=0.25, random_state=12345)
display(df_train.shape)
display(df_test.shape)
display(df_valid.shape)

(1928, 5)

(643, 5)

(643, 5)

In [6]:
# создадим для каждой выборки признаки и ответы
features_train = df_train.loc[:,'calls':'mb_used']
target_train = df_train['is_ultra']

features_valid = df_valid.loc[:,'calls':'mb_used']
target_valid = df_valid['is_ultra']

features_test = df_test.loc[:,'calls':'mb_used']
target_test = df_test['is_ultra']

## Исследуйте модели

Проверим следующие модели: 
- дерево решений (классификация)
- случайный лес (классификация)
- логистическая регрессия 

Берем на проверку именно эти модели, так как у нас категориальные переменные в целевом признаке. Дерево решений в регрессии, случайный лес в регрессии и линейную регрессию не будем рассматривать, так как в них проверяется количественный целевой признак.

**Дерево решений**

In [7]:
# проверим, при какой глубине лучшие результаты
list_depth = []
list_values = []

for i in range(1,6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=i)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid) # получите предсказания модели
    list_depth.append(i)
    list_values.append(accuracy_score(target_valid, predictions_valid))
    
depth_dict = {'max_depth' : list_depth, 'values': list_values}
depth_table = pd.DataFrame(depth_dict)
display(depth_table)

max_depth_result = depth_table['values'].max()

print('Наилучший результат у модели:')
print(depth_table[depth_table['values'] == max_depth_result])

,max_depth,values
0,1,0.748056
1,2,0.783826
2,3,0.786936
3,4,0.786936
4,5,0.788491


Наилучший результат у модели:
   max_depth    values
4          5  0.788491


**Случайный лес**

In [20]:
# проверим, при каком количестве деревьев лучшие результаты
best_model = None
best_result = 0
for est in range(1, 11): 
    for depth in range(1, 6): 
        for samples_leaf in range(1, 6): 
            model = RandomForestClassifier(random_state=12345, n_estimators=est, min_samples_leaf=samples_leaf)
            model.fit(features_train, target_train)
            result = model.score(features_valid, target_valid)
            if result > best_result:
                best_model = model
                best_result = result
print("Accuracy наилучшей модели на валидационной выборке:", best_result)

Accuracy наилучшей модели на валидационной выборке: 0.7869362363919129


**Логистическая регрессия**

In [ ]:
# проверим, при каком количестве деревьев лучшие результаты
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
result = model.score(features_valid, target_valid)

print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Из трех моделей лучший результат показала модель дерева решений с максимальной глубиной 5. Проверим ее на тестовой выборке

## Проверка модели на тестовой выборке

In [21]:
model = DecisionTreeClassifier(random_state=12345, max_depth=5)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test) # получите предсказания модели
print(accuracy_score(target_test, predictions_test))

0.7589424572317263


## Проверка модели на адекватность

In [22]:
model_dummy = DummyClassifier(strategy="most_frequent")
model_dummy.fit(features_train, target_train)
DummyClassifier(strategy='most_frequent')
predictions_test = model_dummy.predict(features_test)
print(model_dummy.score(predictions_test, target_test))

0.6889580093312597


## Вывод

- **Цель проекта**

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».Нужно построить модель для задачи классификации, которая выберет подходящий тариф. 

- **Полученные данные**

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Предобработка не требовалась.

- **Выборки**

Разделили данные на три выборки: обучающую, валидационную и тестовую. От общих данных обучающая составила 60%, валидационная и тестовая по 20%.

- **Исследование разных моделей**

Были рассмотрены 3 модели для категориального целевого признака:

    * дерево решений с параметрами глубины дерева
    * случайный лес с параметрами количества деревьев и количества листьев
    * логистическая регрессия 

Лучше всех себя показала модель дерева решений, ее accuracy_score составил более 78%

- **Проверка модели на тестовой выборке **

Проверили модель решающего дерева на тестовой выборке, результат accuracy_score составил более 75%

- **Проверка модели на адекватность**

С помощью DummyClassifier проверили модель на адекватность, score  составил более 68%

- **Общий вывод**

С помощью исследования построили модель решающего дерева, которая лучше других справилась с задачей классификации, следовательно, ее стоит внедрить для:

    * успешного анализа поведения клиентов
    * предложения им выгодного тарифа, исходя из поведения клиентов